In [1]:
%load_ext autoreload
%autoreload 2

## Data preparation

In [2]:
from importlib.resources import files

import pandas as pd
import numpy as np

from biobank_olink.dataset import load_datasets

DATA_DIR = files("biobank_olink.data")

ol_df, cov_df = load_datasets(cols_na_th=0, rows_na_th=0)
print("ol_df shape: {}".format(ol_df.shape))
print("cov_df shape: {}".format(cov_df.shape))

ol_df shape: (39674, 1463)
cov_df shape: (39674, 10)


In [3]:
cov_df = cov_df.loc[~(cov_df.BMI.isna() | cov_df.SBP.isna())]
cov_df = cov_df[cov_df.HTNgroup < 2]
ol_df = ol_df.loc[cov_df.index]
print("ol_df shape: {}".format(ol_df.shape))
print("cov_df shape: {}".format(cov_df.shape))

ol_df shape: (30644, 1463)
cov_df shape: (30644, 10)


## Evaluation

In [4]:
olink_assays = pd.read_csv(DATA_DIR / "olink-explore-3072-assay-list-2023-06-08.csv")
olink_assays["Explore 384 panel"] = olink_assays.loc[:, "Explore 384 panel"].apply(lambda x: x.split("_")[0])
assays_mapping = olink_assays.groupby("Explore 384 panel")["Gene name"].apply(list).to_dict()
list(assays_mapping.keys())

['Cardiometabolic', 'Inflammation', 'Neurology', 'Oncology']

In [5]:
from sklearn.model_selection import cross_val_score
from xgboost import XGBRegressor

baseline_cols = ["Sex", "age", "BMI"]


def experiment(x, y, fit_params=None):
    model = XGBRegressor(tree_method="gpu_hist", random_state=42)
    scores = cross_val_score(model, x, y, scoring="r2", cv=5, n_jobs=5)
    return scores


scores = {}
x_base = cov_df[baseline_cols]

scores["baseline"] = experiment(x_base, cov_df.SBP)

scores[f"all ({ol_df.shape[1]})"] = experiment(x_base.join(ol_df), cov_df.SBP)

for assay_name in assays_mapping.keys():
    cols = [c for c in ol_df.columns if c in assays_mapping[assay_name]]
    exp_score = experiment(x_base.join(ol_df.loc[:, cols]), cov_df.SBP)
    scores[f"{assay_name.lower()} ({len(cols)})"] = exp_score

In [6]:
scores_df = pd.DataFrame(scores).mean().to_frame("r2")
scores_df["corr"] = np.sqrt(scores_df["r2"])
scores_df

,r2,corr
baseline,0.129290,0.359569
all (1463),0.280571,0.529690
cardiometabolic (367),0.181723,0.426290
inflammation (409),0.240532,0.490441
neurology (347),0.163991,0.404959
oncology (349),0.159223,0.399028
